In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)
plt.scatter(X,y)
plt.axis([0, 2, 0, 15])
plt.ylabel("$y$", rotation=0, fontsize=18)       #$sign just making it itlaics

X_b = np.c_[np.ones((100, 1)), X]  # add x0 = 1 to each instance
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

X_new = np.array([[0], [2]])
X_new_b = np.c_[np.ones((2, 1)), X_new]  # add x0 = 1 to each instance
y_predict = X_new_b.dot(theta_best)

plt.plot(X_new, y_predict, "r-")
plt.plot(X, y, "b.")
plt.axis([0, 2, 0, 15])
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X, y)
lin_reg.intercept_, lin_reg.coef_
lin_reg.predict(X_new)

In [ ]:
#understanding batch descent vs Stochastic gradient descend
eta = 0.1
n_iterations = 1000
m = 100
theta = np.random.randn(2,1)

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients

    
#stochastic from scratch
theta_path_sgd = []
m = len(X_b)
np.random.seed(42)
n_epochs = 50
t0, t1 = 5, 50  # learning schedule hyperparameters

def learning_schedule(t):
    return t0 / (t + t1)

theta = np.random.randn(2,1)  # random initialization

for epoch in range(n_epochs):
    for i in range(m):
        if epoch == 0 and i < 20:                    # not shown in the book
            y_predict = X_new_b.dot(theta)           # not shown
            style = "b-" if i > 0 else "r--"         # not shown
            plt.plot(X_new, y_predict, style)        # not shown
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta * gradients
        theta_path_sgd.append(theta)                 # not shown

plt.plot(X, y, "b.")                                 # not shown
plt.xlabel("$x_1$", fontsize=18)                     # not shown
plt.ylabel("$y$", rotation=0, fontsize=18)           # not shown
plt.axis([0, 2, 0, 15])                              # not shown                          # not shown
plt.show()  

#-----------builtin for stochastic
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(max_iter=50, tol=-np.infty, penalty=None, eta0=0.1, random_state=42)
sgd_reg.fit(X, y.ravel())
sgd_reg.intercept_, sgd_reg.coef_

#-------mini batch descent


theta_path_mgd = []

n_iterations = 50
minibatch_size = 20

np.random.seed(42)
theta = np.random.randn(2,1)  # random initialization

t0, t1 = 200, 1000
def learning_schedule(t):
    return t0 / (t + t1)

t = 0
for epoch in range(n_iterations):
    shuffled_indices = np.random.permutation(m)
    X_b_shuffled = X_b[shuffled_indices]
    y_shuffled = y[shuffled_indices]
    for i in range(0, m, minibatch_size):
        t += 1
        xi = X_b_shuffled[i:i+minibatch_size]
        yi = y_shuffled[i:i+minibatch_size]
        gradients = 2/minibatch_size * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(t)
        theta = theta - eta * gradients
        theta_path_mgd.append(theta)



In [ ]:
#generating quadratic tdata for polynomial regression
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1)
plt.scatter(X, y)

from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)

lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
lin_reg.intercept_, lin_reg.coef_

In [ ]:
#for plotting the regressor curve of different models
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

for style, width, degree in (("g-", 1, 300), ("b--", 2, 2), ("r-+", 2, 1)):
    polybig_features = PolynomialFeatures(degree=degree, include_bias=False)
    std_scaler = StandardScaler()
    lin_reg = LinearRegression()
    polynomial_regression = Pipeline([
            ("poly_features", polybig_features),
            ("std_scaler", std_scaler),
            ("lin_reg", lin_reg),
        ])
    polynomial_regression.fit(X, y)
    y_newbig = polynomial_regression.predict(X_new)
    plt.plot(X_new, y_newbig, style, label=str(degree), linewidth=width)

plt.plot(X, y, "b.", linewidth=3)
plt.legend(loc="upper left")
plt.xlabel("$x_1$", fontsize=18)
plt.ylabel("$y$", rotation=0, fontsize=18)
plt.axis([-3, 3, 0, 10])
plt.show()


In [ ]:
#plotting learing curves (first is underfitting because rmse is high and train/test is same)

# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import train_test_split

# def plot_learning_curves(model, X, y):
#     X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=10)
#     train_errors, val_errors = [], []
#     for m in range(1, len(X_train)):
#         model.fit(X_train[:m], y_train[:m])
#         y_train_predict = model.predict(X_train[:m])
#         y_val_predict = model.predict(X_val)
#         train_errors.append(mean_squared_error(y_train[:m], y_train_predict))
#         val_errors.append(mean_squared_error(y_val, y_val_predict))

#     plt.plot(np.sqrt(train_errors), "r-+", linewidth=2, label="train")
#     plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="val")
#     plt.legend(loc="upper right", fontsize=14)   # not shown in the book
#     plt.xlabel("Training set size", fontsize=14) # not shown
#     plt.ylabel("RMSE", fontsize=14)              # not shown

# lin_reg = LinearRegression()
# plot_learning_curves(lin_reg, X, y)
# plt.axis([0, 80, 0, 3])                         # not shown in the book
# plt.show()  


#overfitting 
from sklearn.pipeline import Pipeline

polynomial_regression = Pipeline([
        ("poly_features", PolynomialFeatures(degree=10, include_bias=False)),
        ("lin_reg", LinearRegression()),
    ])

plot_learning_curves(polynomial_regression, X, y)
plt.axis([0, 80, 0, 3])           # not shown

plt.show()  

In [ ]:
#regularizations

# from sklearn.linear_model import Ridge
# ridge_reg = Ridge(alpha=1, solver="cholesky", random_state=42)
# ridge_reg.fit(X, y)
# ridge_reg.predict([[1.5]])

# d_reg = SGDRegressor(max_iter=50, tol=-np.infty, penalty="l2", random_state=42)
# sgd_reg.fit(X, y.ravel())
# sgd_reg.predict([[1.5]])

# #lasso

# from sklearn.linear_model import Lasso
# lasso_reg = Lasso(alpha=0.1)
# lasso_reg.fit(X, y)
# lasso_reg.predict([[1.5]])


# d_reg = SGDRegressor(max_iter=50, tol=-np.infty, penalty="l1", random_state=42)
# sgd_reg.fit(X, y.ravel())
# sgd_reg.predict([[1.5]])
 
# #Elastic net
# from sklearn.linear_model import ElasticNet
# elastic_net = ElasticNet(alpha = 0.1, l1_ratio = 0.5)
# elastic_net.fit(X,y)
# elastic_net.predict([[1.5]])

#Early stopping
from sklearn.base import clone
np.random.seed(42)


m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 2 + X + 0.5 * X**2 + np.random.randn(m, 1)

X_train, X_val, y_train, y_val = train_test_split(X[:50], y[:50].ravel(), test_size=0.5, random_state=10)

poly_scaler = Pipeline([
        ("poly_features", PolynomialFeatures(degree=90, include_bias=False)),
        ("std_scaler", StandardScaler()),
    ])

X_train_poly_scaled = poly_scaler.fit_transform(X_train)
X_val_poly_scaled = poly_scaler.transform(X_val)

sgd_reg = SGDRegressor(max_iter=1, tol=-np.infty, penalty=None,  eta0=0.0005, warm_start=True,learning_rate="constant",
                       random_state=42)


minimum_val_error = float("inf")
best_epoch = None
best_model = None

n_epochs = 1000
train_errors, val_errors = [], []
for epoch in range(n_epochs):
    sgd_reg.fit(X_train_poly_scaled, y_train)
    y_train_predict = sgd_reg.predict(X_train_poly_scaled)
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)
    val_error = mean_squared_error(y_val, y_val_predict)
    
    train_errors.append(mean_squared_error(y_train, y_train_predict))
    val_errors.append(val_error)
    if val_error < minimum_val_error:
        minimum_val_error = val_error
        best_epoch = epoch
        best_model = clone(sgd_reg)

#best_epoch = np.argmin(val_errors)
best_val_rmse = np.sqrt(val_errors[best_epoch])

plt.annotate('Best model',
             xy=(best_epoch, best_val_rmse),
             xytext=(best_epoch, best_val_rmse + 1),
             ha="center",
             arrowprops=dict(facecolor='black', shrink=0.05),
             fontsize=16,
            )

best_val_rmse -= 0.03  # just to make the graph look better
plt.plot([0, n_epochs], [best_val_rmse, best_val_rmse], "k:", linewidth=2)
plt.plot(np.sqrt(val_errors), "b-", linewidth=3, label="Validation set")
plt.plot(np.sqrt(train_errors), "r--", linewidth=2, label="Training set")
plt.legend(loc="upper right", fontsize=14)
plt.xlabel("Epoch", fontsize=14)
plt.ylabel("RMSE", fontsize=14)

plt.show()

best_model

In [ ]:
#plotting sigmoid

t = np.linspace(-10, 10, 100)
sig = 1 / (1 + np.exp(-t))
plt.figure(figsize=(9, 3))
plt.plot([-10, 10], [0, 0], "k-")
plt.plot([-10, 10], [0.5, 0.5], "k:")
plt.plot([-10, 10], [1, 1], "k:")
plt.plot([0, 0], [-1.1, 1.1], "k-")
plt.plot(t, sig, "b-", linewidth=2, label=r"$\sigma(t) = \frac{1}{1 + e^{-t}}$")
plt.xlabel("t")
plt.legend(loc="upper left", fontsize=20)
plt.axis([-10, 10, -0.1, 1.1])
plt.show()



In [ ]:
#playing with iris dataset

from sklearn import datasets
iris = datasets.load_iris()
list(iris.keys())



In [ ]:
#Decision boundaries
X =  iris['data'][:, 3:]
y = (iris['target'] == 2).astype(int)

from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver="liblinear",random_state=42)
log_reg.fit(X, y)

X_new = np.linspace(0, 3, 1000).reshape(-1, 1)
y_proba = log_reg.predict_proba(X_new)

plt.plot(X_new, y_proba[:, 1], "g-", linewidth=2, label="Iris-Virginica")
plt.plot(X_new, y_proba[:, 0], "b--", linewidth=2, label="Not Iris-Virginica")

#to see where the line is for the two classes
y_proba = log_reg.predict_proba(X_new)
decision_boundary = X_new[y_proba[:, 1] >= 0.5][0]

log_reg.predict([[1.7], [1.5]])



In [30]:
#softmax regression

X = iris["data"][:, (2, 3)]  # petal length, petal width
y = iris["target"]

softmax_reg = LogisticRegression(multi_class="multinomial",solver="lbfgs", C=10, random_state=42)
softmax_reg.fit(X, y)

softmax_reg.predict_proba([[5,2]])


NameError: name 'LogisticRegression' is not defined

In [34]:
#question 12
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn import datasets
iris = datasets.load_iris()



def hotcoding_labels(labels):
    nar = np.zeros((labels.shape[0], n_classes))
    nar[np.arange(labels.shape[0]), labels] = 1
    return nar

def sfmax_func(array_of_scores):
    array_exp = np.exp(array_of_scores)                         #shape 150, 3
    ar_row_sum = np.sum(array_exp, axis = 1, keepdims = True)   #shape = 150, 1
    pk = array_exp / ar_row_sum
    return pk



X = iris["data"][:, (2, 3)]  # petal length, petal width
y_all = iris["target"]
eta = 0.1 

num_samples = X.shape[0]
val_num = np.ceil(0.2 * num_samples).astype(int)
train_num = np.ceil(0.6 * num_samples).astype(int)

shuffle_indices = np.random.permutation(X.shape[0])
X1 = X[shuffle_indices[:train_num]]
X_val = X[shuffle_indices[train_num: train_num + val_num]]
X_test = X[shuffle_indices[train_num + val_num :]]
y = y_all[shuffle_indices[:train_num]]
y_val = y_all[shuffle_indices[train_num: train_num + val_num]]
y_test = y_all[shuffle_indices[train_num + val_num :]]

ep = 1e-10
m = y.shape[0]
X_train = np.c_[np.ones((m, 1)), X1]  # add 1 to each instance at the beginning
X_val = np.c_[np.ones((X_val.shape[0],1)), X_val]
X_test = np.c_[np.ones((X_test.shape[0],1)), X_test]

n_classes = len(np.unique(y))
n_feats = X_train.shape[1]

#we need to change the target label in a way that is understood by the loss equation
labels_encoded = hotcoding_labels(y)
labels_encoded_val = hotcoding_labels(y_val)

thetas = np.random.rand(n_feats ,n_classes)

condition_stop = 100 
count = 0
ite = 0
min_val_error = float('inf')
num_iterations = 5000

while(count < condition_stop and ite < num_iterations):

    ite = ite + 1
    #print('iteration num: ',ite)
    sm_score_train = np.dot(X_train, thetas)
    pk_train = sfmax_func(sm_score_train)

    sm_score_val = np.dot(X_val, thetas)
    pk_val = sfmax_func(sm_score_val)
    
    #val_out = np.argmax(pk_val, axis = 1)
    #val_error = mean_squared_error(y_val, val_out)
    #print('val_error', val_error)
    jtheta_val = -np.mean(np.sum(labels_encoded_val * np.log(pk_val + ep), axis=1))
    #print('validation array out: ', val_out)
    
    if jtheta_val < min_val_error:
        print('val_error < min_val_error')
        min_val_error = val_error
        min_thetas = thetas
        count = 0
    else:
        count += 1
    
    #print('count: ',count)
    Jtheta = -np.mean(np.sum(labels_encoded * np.log(pk_train + ep), axis=1))
    print('Training',ite, Jtheta)
    print('validation',ite, jtheta_val)
    
    error = pk_train - labels_encoded
    newar = []
    for i in range(n_classes):
        er = error[:, i].reshape(-1, 1)
        toappend = np.mean((X_train * er), axis = 0)
        newar.append(toappend)
        gradients = np.array(newar).T

    #gradeints = gradients.T   #becuase my way resulted in classes in rows as in 1st row first class theta second row second class theta and so on. taking transpose makes it how it is 
    thetas = thetas - eta * gradients

#loop ends

val_error < min_val_error
Training 1 1.238415601320209
validation 1 1.3311828530753462
Training 2 1.0907866085391469
validation 2 1.1483093347688482
Training 3 1.0590579673088991
validation 3 1.1046972155253536
Training 4 1.0484323614849465
validation 4 1.093273566232311
Training 5 1.0412123219979261
validation 5 1.0867060492599063
Training 6 1.0351043783138425
validation 6 1.080847978245178
Training 7 1.029472788661741
validation 7 1.0750350791883
Training 8 1.0240772624648897
validation 8 1.0691688473038394
Training 9 1.018817498092705
validation 9 1.0632526376468465
Training 10 1.013649084584223
validation 10 1.0573081997356089
Training 11 1.008551496641026
validation 11 1.051357592434968
Training 12 1.0035148926690582
validation 12 1.0454190144300586
Training 13 0.9985343619024053
validation 13 1.0395063621214236
Training 14 0.9936073192419695
validation 14 1.0336298303972173
Training 15 0.9887322929088864
validation 15 1.0277967087269677
Training 16 0.9839083491917767
validation 1

In [35]:
#now lets see the accuracy on training and validations sets

thetas = min_thetas
sm_score_train = np.dot(X_train, thetas)
pk_train = sfmax_func(sm_score_train)
y_pred = np.argmax(pk_train, axis = 1)



In [24]:
np.mean(y_pred == y)

0.7222222222222222

In [ ]:
nar = np.zeros((m, n_classes))
nar[np.arange(m), y] = 1
nar